# Partie 1

## Modules

In [2]:
import openpyxl
from classes import Ressource,RessourceUnavailability,Task,TaskUnavailability
from read_input import read_ressources,read_ressources_unavailabilities,read_tasks,read_tasks_unavailabilities

from gurobipy import *
import math

## Lecture des données

In [2]:
ressources = read_ressources(1)
ressources_unavailabilities = read_ressources_unavailabilities(1)
tasks = read_tasks(1)
tasks_unavailabilities = read_tasks_unavailabilities(1)

44.556549383420084 -0.31939224223757195 60 Oenology 1 8:00am 6:00pm
44.967500952177986 -0.6086852638150881 60 Oenology 1 8:00am 6:00pm
45.14421541464031 -0.7342570469020379 60 Oenology 2 8:00am 6:00pm
45.264808304867096 -0.7717887212411139 60 Oenology 1 8:00am 6:00pm
45.044422793402624 -0.6687606009488057 60 Oenology 2 8:00am 6:00pm
45.19957452440505 -0.7462077931750715 60 Oenology 2 8:00am 6:00pm
45.397697776585 -0.9668192708194538 60 Oenology 1 8:00am 6:00pm
45.023479086796385 -0.8072126299796225 60 Oenology 1 8:00am 6:00pm
45.29291368453335 -0.9365361007032235 60 Oenology 2 8:00am 6:00pm
45.08146166752168 -0.8062453230620741 60 Oenology 1 8:00am 6:00pm


## Structure de données

In [33]:
def dist(lat1, long1, lat2, long2):
    return(math.sqrt((lat1-lat2)**2+(long1-long2)**2))

v=833.3333333333333

L_ressources = [j for j in ressources]
L_tasks = [i for i in tasks]
number_of_ressources = len(L_ressources)
number_of_tasks = len(L_tasks)
dict_pos = {task : (float(task.latitude), float(task.longitude)) for task in tasks.values()}

dict_pos = {task : (float(tasks[L_tasks[task]].latitude), float(tasks[L_tasks[task]].longitude)) for task in range(number_of_tasks)}
#graph = {(task1,task2): dist(float(tasks[L_tasks[task1].latitude]), float(tasks[L_tasks[task1].longitude]), float(tasks[L_tasks[task2].latitude]), float(tasks[L_tasks[task2].longitude])) for task1 in range(number_of_tasks) for task2 in range(number_of_tasks)}
print(dict_pos)

# creer locations
# creer dists

# https://gurobi.github.io/modeling-examples/technician_routing_scheduling/technician_routing_scheduling.html


{0: (44.556549383420084, -0.31939224223757195), 1: (44.967500952177986, -0.6086852638150881), 2: (45.14421541464031, -0.7342570469020379), 3: (45.264808304867096, -0.7717887212411139), 4: (45.044422793402624, -0.6687606009488057), 5: (45.19957452440505, -0.7462077931750715), 6: (45.397697776585, -0.9668192708194538), 7: (45.023479086796385, -0.8072126299796225), 8: (45.29291368453335, -0.9365361007032235), 9: (45.08146166752168, -0.8062453230620741)}


## Modèle

In [34]:
### Create Model
m = Model("trs0")

### Decision variables
# Task-ressource assignment
x = {(t,r) : m.addVar(vtype = GRB.BINARY, name = f'x_{t}_{r}') for r in range(number_of_ressources) for t in range(number_of_tasks)}

# Time-task assignment
s = {t : m.addVar(vtype = GRB.INTEGER, lb = 0, ub = 1440, name = f's_{t}') for t in range(number_of_tasks)}
# Les s_t valent entre 0 et 1440 car les contraintes de temps (entre 8h et 18h) sont contenues dans les jeux de données.

# Est-ce que le travailleur se déplace entre les points a et b ?
y = {(r,a,b) : m.addVar(vtype = GRB.BINARY, name = f'y_{r}_{a}_{b}') for r in range(number_of_ressources) for a in range(number_of_tasks) for b in range(number_of_tasks)}# if a != b}
# Technician cannot leave or return to a depot that is not its base

In [37]:
nbRpT = [LinExpr() for t in range(number_of_tasks)]
for t in range(number_of_tasks):
  for r in  range(number_of_ressources):
    nbRpT[t]+=x[(t,r)]

# One ressource per task
C1 = [m.addConstr(nbRpT[t] <= 1) for t in range(number_of_tasks)]

#  Works only if all tasks are feasable
# nbRpTq = {t: LinExpr() for t in range(number_of_tasks)}
# for t in range(number_of_tasks):
#   for r in  range(number_of_ressources):
#     if ressources[r].skill==tasks[t].skill and ressources[r].level>=tasks[t].level:
#       nbRpTq[t]+=x[(t,r)]

# # Qualified technicians
# C2 = [m.addConstr(nbRpTq[t] >= 1) for t in range(number_of_tasks)]

nbRpTs = [LinExpr() for t in range(number_of_tasks)]
for t in range(number_of_tasks):
  for r in  range(number_of_ressources):
    nbRpTs[t]+=x[(t,r)]*(ressources[L_ressources[r]].skill!=tasks[L_tasks[r]].skill)

# Skilled technicians
C2 = [m.addConstr(nbRpTs[t] == 0) for t in range(number_of_tasks)]

nbRpTl = [LinExpr() for t in range(number_of_tasks)]
for t in range(number_of_tasks):
  for r in  range(number_of_ressources):
    nbRpTl[t]+=x[(t,r)]*(ressources[L_ressources[r]].level-tasks[L_tasks[r]].level)

# Leveled technicians
C3 = [m.addConstr(nbRpTl[t] >= 0) for t in range(number_of_tasks)]

# si une personne est affectée, elle arrive jusqu'à la tâche
C4 = {(i,j) : m.addConstr(quicksum(y[(j,a,i)] for a in range(number_of_tasks)) == x[(i,j)], name = f'arrive{i}{j}') for j in range(number_of_ressources) for i in range(number_of_tasks)}

# une personne doit partir après avoir exécuté sa tâche
C5 = {(i,j) : m.addConstr(quicksum(y[(j,i,b)] for b in range(number_of_tasks)) == x[(i,j)], name = f'part{i}{j}') for j in range(number_of_ressources) for i in range(number_of_tasks)}

In [4]:
# -- Ajout de la fonction objectif --
cout=LinExpr()
for j in range(number_of_ressources):
  for a in range(number_of_tasks):
    for b in range(number_of_tasks):
      cout+=y[(j,a,b)]*d
m.setObjective(1, GRB.MINIMIZE)

-1


In [ ]:
# -- Choix d'un paramétrage d'affichage minimaliste --
m.params.outputflag = 0 # mode muet

# -- Mise à jour du modèle  --
m.update()

# -- Affichage en mode texte du PL --
m.display()

# -- Résolution --
m.optimize()

In [ ]:
#Output